In [1]:
import data

Using TensorFlow backend.


In [2]:
from concise.preprocessing import EncodeSplines

In [59]:
dt = pd.read_csv(data.DATA_ROOT + "/Splice_branchpoints/processed/branchpointer/train/filteredDescr.csv")
dt_test = pd.read_csv(data.DATA_ROOT + "/Splice_branchpoints/processed/branchpointer/test/filteredDescr.csv")

In [29]:
dt

,set,dist.1,dist.2,ppt_start,ppt_run_length,canon_hit1,canon_hit2,canon_hit3,canon_hit4,canon_hit5,seq_neg5A,seq_neg5C,seq_neg5G,seq_neg5T,seq_neg4A,seq_neg4C,seq_neg4G,seq_neg4T,seq_neg3A,seq_neg3C,seq_neg3G,seq_neg3T,seq_neg2A,seq_neg2C,seq_neg2G,seq_neg2T,seq_neg1A,seq_neg1C,seq_neg1G,seq_neg1T,seq_pos0A,seq_pos0C,seq_pos0G,seq_pos0T,seq_pos1A,seq_pos1C,seq_pos1G,seq_pos1T,seq_pos2A,seq_pos2C,seq_pos2G,seq_pos2T,seq_pos3A,seq_pos3C,seq_pos3G,seq_pos3T,seq_pos4A,seq_pos4C,seq_pos4G,seq_pos4T,seq_pos5A,seq_pos5C,seq_pos5G,seq_pos5T
0,HC,8726,16,5,42,52,122,159,201,229,1,1,1,2,1,2,1,1,1,2,1,1,1,1,1,2,1,2,1,1,2,1,1,1,1,2,1,1,2,1,1,1,1,1,1,2,1,1,1,2,1,1,1,2
1,HC,7104,15,35,44,51,127,151,181,199,1,1,1,2,2,1,1,1,1,1,2,1,1,1,1,2,1,1,2,1,2,1,1,1,1,2,1,1,1,1,1,2,1,1,1,2,2,1,1,1,2,1,1,1
2,HC,4725,4,13,11,38,122,156,212,14,1,2,1,1,1,1,1,2,1,1,2,1,1,1,1,2,1,2,1,1,2,1,1,1,1,2,1,1,1,1,1,2,1,1,1,2,1,1,1,2,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734193,NEG,12980,25,19,44,1,184,78,168,200,1,2,1,1,2,1,1,1,1,2,1,1,2,1,1,1,1,1,2,1,2,1,1,1,2,1,1,1,1,1,2,1,1,1,1,2,1,1,2,1,1,2,1,1
734194,NEG,12969,26,20,44,39,1,89,169,201,1,1,2,1,1,2,1,1,2,1,1,1,1,2,1,1,2,1,1,1,1,1,2,1,2,1,1,1,2,1,1,1,1,1,2,1,1,1,1,2,1,1,2,1
734195,NEG,12958,27,21,44,50,12,100,170,202,1,1,1,2,1,1,2,1,1,2,1,1,2,1,1,1,1,2,1,1,2,1,1,1,1,1,2,1,2,1,1,1,2,1,1,1,1,1,2,1,1,1,1,2


In [3]:
def read_long_data(n_bases=None):
    dt = pd.read_csv(data.DATA_ROOT + "/Splice_branchpoints/processed/branchpointer/train/filteredDescr.csv")
    dt_test = pd.read_csv(data.DATA_ROOT + "/Splice_branchpoints/processed/branchpointer/test/filteredDescr.csv")

    del dt["Unnamed: 0"]
    del dt_test["Unnamed: 0"]

    y_train = dt.set == "HC"
    y_test = dt_test.set == "HC"

    x_seq_train = dt.loc[:, dt.columns.str.match("^seq")]
    x_seq_test = dt_test.loc[:, dt_test.columns.str.match("^seq")]

    x_pos_train = dt.loc[:, ~dt.columns.str.match("^seq|^set$")]
    x_pos_test = dt_test.loc[:, ~dt_test.columns.str.match("^seq|^set$")]

    if n_bases is not None:
        pos_features = list(x_pos_train.columns)

        assert np.all(x_pos_train.columns == x_pos_test.columns)

        x_pos_train = x_pos_train.as_matrix()
        x_pos_test = x_pos_test.as_matrix()

        st = EncodeSplines(n_bases=n_bases)

        st.fit(x_pos_train)

        x_pos_bs_train = st.transform(x_pos_train)
        x_pos_bs_test = st.transform(x_pos_test)

        x_pos_bs_train = x_pos_bs_train.reshape((x_pos_bs_train.shape[0], -1))
        x_pos_bs_test = x_pos_bs_test.reshape((x_pos_bs_test.shape[0], -1))


        x_train = np.concatenate([x_seq_train, x_pos_bs_train], axis = 1)
        x_test = np.concatenate([x_seq_test, x_pos_bs_test], axis = 1)
        return (x_train, y_train, pos_features, st), (x_test, y_test)
    else:
        return (np.array(x_seq_train), np.array(y_train)), (np.array(x_seq_test), np.array(y_test))
        


In [21]:
train, test = read_long_data(10)

In [118]:
train[0].shape

(734196, 134)

In [5]:
train_seqonly, test_seqonly = read_long_data(None)

In [6]:
train_seqonly[0].shape

(734196, 44)

## Sequence + distance model

In [9]:
from glmnet import LogitNet

In [15]:
ln = LogitNet(alpha=0.5, n_splits=10, n_jobs=5)

In [16]:
from scipy.sparse import csc_matrix

In [73]:
# to sparse
train_sparse = (csc_matrix(train[0]), csc_matrix(train[1].values.astype(np.float64).reshape((-1,1))))
test_sparse = (csc_matrix(test[0]), csc_matrix(test[1].values.astype(np.float64).reshape((-1,1))))

In [94]:
# to sparse
train_sparse_seqonly = (csc_matrix(train_seqonly[0]), csc_matrix(train_seqonly[1].astype(np.float64).reshape((-1,1))))
test_sparse_seqonly = (csc_matrix(test_seqonly[0]), csc_matrix(test_seqonly[1].astype(np.float64).reshape((-1,1))))

In [66]:
train[1]

0          True
1          True
2          True
          ...  
734193    False
734194    False
734195    False
Name: set, Length: 734196, dtype: bool

In [32]:
train_sparse[1].shape

(1, 734196)

In [ ]:
train[0] = 

In [87]:
%time ln.fit(train_sparse[0], train[1])

CPU times: user 2h 56min 27s, sys: 16.2 s, total: 2h 56min 43s
Wall time: 39min 35s


LogitNet(alpha=0.5, cut_point=1.0, fit_intercept=True, lambda_path=None,
     max_iter=100000, min_lambda_ratio=0.0001, n_jobs=5, n_lambda=100,
     n_splits=10, random_state=None, scoring=None, standardize=True,
     tol=1e-07, verbose=False)

In [88]:
y_pred = ln.predict_proba(test_sparse[0])[:, 1]

In [89]:
y_true = test[1]

In [80]:
import concise.eval_metrics as cem

In [90]:
cem.auprc(y_true, y_pred)

0.48468812870060429

In [100]:
cem.auc(y_true, y_pred)

0.90138928107495186

## Model from the plot

In [272]:
load_dir = "../../data/Splice_branchpoints/processed/pr_roc/"

In [274]:
roc_curves = pd.read_csv(os.path.join(load_dir, "roc_curves.csv"))
pr_curves = pd.read_csv(os.path.join(load_dir, "pr_curves.csv"))

In [278]:
pr_curves[pr_curves.method == "glmnet"]

,Recall,Precision,cut,method,auPR,method_perf
765711,1.0000e+00,0.0573,-10.0043,glmnet,0.5022,"glmnet, 0.502"
765712,1.0000e+00,0.0573,-10.0043,glmnet,0.5022,"glmnet, 0.502"
765713,1.0000e+00,0.0573,-9.9527,glmnet,0.5022,"glmnet, 0.502"
...,...,...,...,...,...,...
1488700,4.7512e-05,1.0000,2.4596,glmnet,0.5022,"glmnet, 0.502"
1488701,2.3756e-05,1.0000,2.5463,glmnet,0.5022,"glmnet, 0.502"
1488702,0.0000e+00,1.0000,2.5463,glmnet,0.5022,"glmnet, 0.502"


In [279]:
# The right accuracy is 0.502

## Sequence-only model

In [95]:
ln_seqonly = LogitNet(alpha=0.5, n_splits=10, n_jobs=5)

In [96]:
%time ln_seqonly.fit(train_sparse_seqonly[0], train_seqonly[1])

CPU times: user 27min 28s, sys: 7.8 s, total: 27min 36s
Wall time: 6min 9s


LogitNet(alpha=0.5, cut_point=1.0, fit_intercept=True, lambda_path=None,
     max_iter=100000, min_lambda_ratio=0.0001, n_jobs=5, n_lambda=100,
     n_splits=10, random_state=None, scoring=None, standardize=True,
     tol=1e-07, verbose=False)

In [97]:
y_pred_seqonly = ln_seqonly.predict_proba(test_sparse_seqonly[0])[:, 1]

In [98]:
cem.auprc(test_seqonly[1], y_pred_seqonly)

0.41880125922995781

In [99]:
cem.auc(test_seqonly[1], y_pred_seqonly)

0.8743892308946899

In [101]:
## iDeep results

### Kmer results

In [104]:
import os

In [173]:
from concise.utils.helper import read_json

In [224]:
PROC_DIR = "../../data/clip/"

In [225]:
proteins = os.listdir(PROC_DIR)

In [226]:
kmer = 6

glob_pattern = PROC_DIR + "evaluation/{protein}/kmer_k={kmer}_pos={pos}.json"

In [190]:
next(listfiles(glob_pattern))

('../../data/clip/evaluation/18_ICLIP_hnRNPL_Hela_group_3975_all-hnRNPL-Hela-hg19_sum_G_hg19--ensembl59_from_2337-2339-741_bedGraph-cDNA-hits-in-genome/kmer_k=6_pos=seq.json',
 ['18_ICLIP_hnRNPL_Hela_group_3975_all-hnRNPL-Hela-hg19_sum_G_hg19--ensembl59_from_2337-2339-741_bedGraph-cDNA-hits-in-genome',
  'seq',
  '6'])

In [177]:
from snakemake.utils import listfiles

In [191]:
def load_file(fname, args):
    arg_names = ["protein", "pos", "kmer"]
    return {**read_json(fname), **dict(zip(arg_names, args))}

In [237]:
ls $PROC_DIR/iDeep_seq_only/10_PARCLIP_ELAVL1A_hg19.csv

../../data/clip//iDeep_seq_only/10_PARCLIP_ELAVL1A_hg19.csv


### Using a sequence only-DNN on iDeep daa

In [238]:
from glob import glob

In [242]:
auprc_list = []
for fname in glob(os.path.join(PROC_DIR, "iDeep_seq_only/*.csv")):
    dt = pd.read_csv(fname)
    auprc_list.append(cem.auprc(dt.y_true, dt.y_pred_proba))

In [245]:
np.array(auprc_list).mean()

0.6252644896091234

### Exclude features

In [210]:
df = pd.DataFrame([load_file(fname, args) for fname, args in listfiles(glob_pattern)])

In [214]:
# fix the labelling
df.pos = df.pos.str.replace("seq+dist", "sequence")

df.pos = df.pos.str.replace("seq$", "sequence + distance", )

In [213]:
df.groupby(["pos"]).mean()

,auc,auprc
pos,,
seq+dist,0.7608,0.5554
sequence + distance,0.8647,0.6602


### Seq-only dnn

### Eclip

In [261]:
DIR_ROOT = os.path.expanduser("~/projects-work/code_spline_trans/data/eclip/")
PROC_DIR = DIR_ROOT + "/processed"
SAVE_ROOT = os.path.join(PROC_DIR, "feature_glmnet_baseline_exp")
glob_pattern = SAVE_ROOT + "/evaluation/{protein}/kmer_k={kmer}_pos={pos}.json"

In [262]:
df = pd.DataFrame([load_file(fname, args) for fname, args in listfiles(glob_pattern)])

In [263]:
df.pos = df.pos.str.replace("seq+dist", "sequence")

df.pos = df.pos.str.replace("seq$", "sequence + distance", )

In [264]:
df

,auc,auprc,kmer,pos,protein
0,0.8172,0.5064,6,sequence + distance,DHX30
1,0.6841,0.3693,6,seq+dist,DHX30
2,0.7953,0.5120,6,seq+dist,ZRANB2
...,...,...,...,...,...
221,0.9092,0.7347,6,sequence + distance,BCCIP
222,0.7699,0.4509,6,seq+dist,MTPAP
223,0.8677,0.6473,6,sequence + distance,MTPAP


In [265]:
df.groupby("pos").mean()

,auc,auprc
pos,,
seq+dist,0.7708,0.5566
sequence + distance,0.8728,0.6773
